In [24]:
#一次处理多个文档，没有replicated文档
#single chain with only tube

import pandas as pd


In [25]:
#1_rearrange
#1_add chain number, time step

#load file
EPS = ['0.20', '0.40', '0.60', '0.80', '1.00', '1.20', '1.40', '1.60', '1.80', '2.00']

for z in EPS:
    fread1 = open('D:\\all\\projects\\python_research_code\\py_count_chains_in_pore\\20230118_NA120_R3_EPS' + z + '\\poly.lammpstrj', 'r')
    longread1 = fread1.readlines()
        
    #strip, split and save data
    longread2 = []
    for j in longread1:
        first = j.strip('/n')
        second = first.split()
        longread2.append(second)

    #delete title in dump file
    #8 means the length of the line that should be used for coding in dump file
    longdump = []

    for j in range(0, len(longread2), 1):
        if len(longread2[j]) == 8:
            longdump.append(longread2[j])

    #add time step
    timesteplong = []
    idumpstep = 200000 # timestep the dump start
    dumpsegment = 100  # dump every time step
    totalparticleslong = 120  # atoms number

    for j in range(1, len(longdump) // totalparticleslong + 1, 1):
        for k in range(1, totalparticleslong + 1, 1):
            timesteplong.append(idumpstep)
        idumpstep = idumpstep + dumpsegment

    #write dump into a new file
    fwrite1 = open('long_rearrange.txt', 'w')
    a = 'ID TYPE x y z ix iy iz ' + '\n'
    fwrite1.write(a)
    for j in range(0, len(longdump), 1):
        c = str(longdump[j]).replace('[','').replace(']','')
        c = c.replace("'",'').replace(',','') + '\n'
        fwrite1.write(c)
    fwrite1.close()

    #write timestep into a new file
    fwrite1 = open('long_time.txt', 'w')
    a = 'timestep' + '\n'
    fwrite1.write(a)
    for j in range(0, len(timesteplong), 1):
        a = str(timesteplong[j]).replace('[', '').replace(']', '')
        a = a.replace("'", '').replace(',', '') + '\n'
        fwrite1.write(a)
    fwrite1.close()
                      
#2_pandas: particles in pore

    #load files
    dfl = pd.read_table('long_rearrange.txt', delim_whitespace=True)
    dfl_t = pd.read_table('long_time.txt', delim_whitespace=True)
    
    #calculate real coordination
    xbox = 50 #the total length of simulation box
    ybox = 50 #the total length of simulation box
    zbox = 101 #the total length of simulation box
    dfl['realx'] = dfl.x + xbox*dfl.ix
    dfl['realy'] = dfl.y + ybox*dfl.iy
    dfl['realz'] = dfl.z + zbox*dfl.iz
        
    #add timestep
    dfl['timestep'] = dfl_t['timestep']
        
    #select particales make contact with pore_wall
    dfl_contact_x = dfl.loc[(dfl['realx']<= -0.5) | (dfl['realx']>= 0.5)]
    dfl_contact_y = dfl_contact_x.loc[(dfl_contact_x['realy']<= -0.5) | (dfl_contact_x['realy']>= 0.5)]
                  
    #write files
    dfl_contact_y.to_csv('long_particles_in_pore.txt', sep=' ', header=None, index=None)

#3_cantact_number_vs_time

    #load file
    freadinital1 = open('long_particles_in_pore.txt', 'r')
    longinitial = freadinital1.readlines()

    #strip and split data
    long = []
    for j in longinitial:
        first = j.strip('/n')
        second = first.split()
        long.append(second)

    #time_+_contact_number
    j = 0
    lcontact = []
    ltimestep = []
    ltime = int(long[0][11])
    for k in range(0, len(long)-1,1):
        if int(long[k][11]) == int(ltime):
            j = j + 1
        else:
            ltimestep.append(ltime)
            lcontact.append(j)
            ltime = int(long[k][11])
            j = 1
            
    #write into a new file
    fwrite1 = open('long_contacts_in_pore_EPS' + z + '.txt', 'w')
    for j in range(0, len(ltimestep), 1):
        s = str(ltimestep[j]).replace('[','').replace(']','')
        s = s.replace("'",'').replace(',','') + ' '
        d = str(lcontact[j]).replace('[','').replace(']','')
        d = d.replace("'",'').replace(',','') +'\n'
        fwrite1.write(s)
        fwrite1.write(d)
    fwrite1.close()


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\all\\projects\\python_research_code\\py_count_chains_in_pore\\20230118_NA120_R3_EPS2.00\\poly.lammpstrj'